For each set of NEs predicted by each package (i.e. Stanza and Spacy), how many of them can be found mentioned in the KG Graph you ollected for the person?7 

return for each package (i.e. Stanza and Spacy), the ratio of
predicted NEs that you can be confidently said to be in the KG
graph for the person.

7 note that you should take into consid-
eration the NE span in the text may not
be an exact match with KG entity label.
Here you can use regular expressions
to match the NEs found in the text
with the RDF entities present in the
associated texts.

In [7]:
import os 
import pandas as pd 
import numpy as np
import json
import re

In [8]:
par_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(par_dir, '2.2 NER - analysis by entity type')

stanza_dir = os.path.join(data_dir, "df_entities_spacy_processed.csv")
spacy_dir = os.path.join(data_dir, "df_entities_stanza_processed.csv")

entities_stanza = pd.read_csv(stanza_dir)
entities_spacy = pd.read_csv(spacy_dir)

In [35]:
entities_spacy[(entities_spacy["text"].str.contains("\(")) & (entities_spacy["text"].str.contains("\)"))]

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\alber\AppData\Local\Temp\ipykernel_2500\4171797603.py:1: SyntaxWarning: invalid escape sequence '\('
  entities_spacy[(entities_spacy["text"].str.contains("\("))]


,text,n_word,start_pos,end_pos,label,subject,source_index,category,range_span
532,Rijksmuseum (National Gallery,3,18017,18046,FAC,Hendrik Lorentz,1,Physics,[18017 18018 18019 18020 18021 18022 18023 180...
2651,Kathodenstrahlen (On Cathode Rays,4,9901,9934,WORK_OF_ART,Philipp Lenard,7,Physics,[9901 9902 9903 9904 9905 9906 9907 9908 9909 ...
3188,Historical Studies in the Physical and Biologi...,9,24287,24350,WORK_OF_ART,J. J. Thomson,8,Physics,[24287 24288 24289 24290 24291 24292 24293 242...
3214,Journal of the American Society for Mass Spect...,10,24759,24820,WORK_OF_ART,J. J. Thomson,8,Physics,[24759 24760 24761 24762 24763 24764 24765 247...
3563,"Bonanza (""Look to the Stars""",5,17213,17241,WORK_OF_ART,Albert A. Michelson,9,Physics,[17213 17214 17215 17216 17217 17218 17219 172...
3844,Lippmann (Centre de Recherche Public Gabriel L...,7,12477,12530,ORG,Gabriel Lippmann,10,Physics,[12477 12478 12479 12480 12481 12482 12483 124...
4224,hundreds of kilometres (miles,4,15510,15539,QUANTITY,Guglielmo Marconi,11,Physics,[15510 15511 15512 15513 15514 15515 15516 155...
4787,"Ahern, Steve (ed",3,33750,33766,ORG,Guglielmo Marconi,11,Physics,[33750 33751 33752 33753 33754 33755 33756 337...
4817,Creative Minds Produced Technology for the Mas...,9,34292,34358,WORK_OF_ART,Guglielmo Marconi,11,Physics,[34292 34293 34294 34295 34296 34297 34298 342...
5786,23(,1,5316,5319,CARDINAL,Gustaf Dalén,15,Physics,[5316 5317 5318 5319]


In [10]:
list_people = list(set(list(np.unique(entities_spacy["subject"])) + list(np.unique(entities_stanza["subject"]))))
list_entities = list(set(list(np.unique(entities_spacy["label"])) + list(np.unique(entities_stanza["label"]))))

In [43]:

def make_regex(entity):
    entity_extra = entity.split(" ")
    entity_new = "_?".join(entity_extra)
    # entity_diff_start_end = "[0-9]{0,2} " + entity +  " [0-9]{0,2} " # to account for dates where there may only be a year in the graph but a full date in the entity
    # entity_underscore = "_?[a-z]{0,9}_?".join(entity.split(" ")) # adding optional underscores and words from length 0-9
    
    return entity_new

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\)'
<>:3: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\)'
<>:5: SyntaxWarning: invalid escape sequence '\('
<>:6: SyntaxWarning: invalid escape sequence '\)'
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\)'
<>:3: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\)'
<>:5: SyntaxWarning: invalid escape sequence '\('
<>:6: SyntaxWarning: invalid escape sequence '\)'
C:\Users\alber\AppData\Local\Temp\ipykernel_2500\2244876200.py:2: SyntaxWarning: invalid escape sequence '\('
  if "\(" in entity and "\)" not in entity:
C:\Users\alber\AppData\Local\Temp\ipykernel_2500\2244876200.py:2: SyntaxWarning: invalid escape sequence '\)'
  if "\(" in entity and "\)" not in entity:
C:\Users\alber\AppData\Local\Temp\ipykernel_2500\2244876200.py:3: SyntaxWarning: invalid esc

In [50]:
for person in list_people[0:3]:
    
    # filtering each df by the subject of the bio
    filter_df_spacy = entities_spacy[entities_spacy["subject"] == person]
    filter_df_stanza = entities_stanza[entities_stanza["subject"] == person]
    
    # getting the category to know which Data Directory subfolder we have to access
    category = filter_df_spacy["category"].iloc[0]
    
    # print(category)
    # getting the entities found in the bios per package
    set_entities_spacy = set(filter_df_spacy["text"].to_list())
    set_entities_stanza = set(filter_df_stanza["text"].to_list())
    
    # total amount of unique entities
    total_entities_spacy = len(set_entities_spacy)
    total_entities_stanza = len(set_entities_stanza)
    
    found_entities_spacy = 0
    found_entities_stanza = 0   
    
    # loading the json graphs to query them
    par_dir = os.path.dirname(os.path.dirname(os.getcwd()))
    name_file_graph = person.replace(" ", "__") + ".json"
    data_dir = os.path.join(par_dir, "Data Directory")
    
    if category == "Chemistry":
        graph_dir = os.path.join(data_dir, "chemistry_nobel_laureate")
    
    else:
        graph_dir = os.path.join(data_dir, "physics_nobel_laureate")
        
    person_graph_path = os.path.join(graph_dir, name_file_graph)
    # print(person_graph_path)
    
    with open(person_graph_path, "r") as f:
        personal_graph = json.load(f)
    # if f:
    #     print("Graph load successfully!")
    
    list_subjects = [dict_graph["subject"]["value"].strip("http://dbpedia.org/resource/") for dict_graph in personal_graph]
    list_objects = [dict_graph["object"]["value"].strip("http://dbpedia.org/resource/") for dict_graph in personal_graph]
    
    list_entities_graph = list(set(list_subjects + list_objects))
    
    # print(list_entities_graph)
    
    regex_entities_stanza = [ent for ent in set_entities_stanza if ent.count("(") == ent.count(")")]
    print(regex_entities_spacy)
    regex_entities_spacy = [ent for ent in set_entities_spacy if ent.count("(") == ent.count(")")]
    
    matches_spacy = 0
    
    print(regex_entities_stanza)
    for ent in list_entities_graph:
        
        match_search = [re.search(regex, ent, flags=re.IGNORECASE) for regex in regex_entities_spacy]
        match = [(search.group(0), search) for search in match_search if search != None]
        print(ent)
        print(match)
        if match:
            matches_spacy = matches_spacy + 1
        
        print(matches_spacy)
    prop_matches = round((matches_spacy/total_entities_spacy), 2)
    
    

['Victor Meyer', 'Wolcott Gibbs Memorial Laboratory', 'International Atomic Weights Committee', 'Theodore William Richards', '1883', 'Harvard University Press.', 'Miriam Stuart Thayer', '1911', '1913', 'National Academy of Sciences', 'Bachelor of Science degree', '1915', 'Harvard University', 'the United States', 'Davy Medal', '1910', '1885', '1917', 'the American Association for the Advancement of Science', 'Cooke', 'two years', 'Bachelor of Arts degree', 'the University of Göttingen', 'Theodore Richards Medal', 'Jakob Berzelius\nFarrington Daniels', 'Josiah Parsons Cooke', 'Massachusetts', 'Le Blanc Medal', 'James Bryant', 'Journal of the American Chemical Society. 37', '1901', '1643', '286', 'Cambridge', 'Member of the American Philosophical Society', 'Washington', 'Newport', '1974', 'Royal Irish Academy', 'Nobelprize.org', '1923', '1656', 'Carnagie Institution', 'William Theodore', '1916', 'John Henry', 'Forbes', '15,000', '1912', 'Richards', '1896', 'Theodore Richards', 'Greenough

In [25]:
help(re.search)

Help on function search in module re:

search(pattern, string, flags=0)
    Scan through string looking for a match to the pattern, returning
    a Match object, or None if no match was found.

